In [1]:
# Импорт необходимых библиотек
import pandas as pd # анализ и обработка данных
import numpy as np # вычислительная мощ языка С
import matplotlib.pyplot as plt # визуализация
import seaborn as sns # расширенная визуализация
#from datetime import datetime # обработка дат и времени
#import re # регулярные выражения
from collections import Counter # словарь-подкласс для подсчета хэш данных
from sklearn.feature_extraction.text import TfidfVectorizer  # Для векторизации текста
from sklearn.model_selection import ( # разделяем данные
    train_test_split,
    cross_val_score,
    StratifiedKFold,
    KFold
)
from sklearn.metrics import (
    fbeta_score, 
    classification_report,
    make_scorer
)
from sentence_transformers import SentenceTransformer # встраивание текста и изображений
from sklearn.linear_model import LogisticRegression # базовая модель
from sklearn.neural_network import MLPClassifier # 
from sklearn.multioutput import MultiOutputClassifier # мульти-лейбл классификация
import joblib # сохранение модели
import warnings # контроль предупреждений
warnings.filterwarnings('ignore') # подавить все не крит. предупреждения

# Импорт для лемматизации и стоп-слов
import spacy
import nltk  # лингвистическая обработки
from nltk.corpus import stopwords  # стоп-слова
from tqdm import tqdm  # прогресс-бары
import pickle

# Скачиваем стоп-слова
try:
    nltk.data.find('corpora/stopwords')  # наличие стоп-слов
except LookupError:
    nltk.download('stopwords')  # скачиваем если отсутствуют

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)
plt.style.use('seaborn-v0_8')

2025-10-06 00:32:52.898329: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759692772.933957   11971 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759692772.985406   11971 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759692773.261928   11971 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759692773.261953   11971 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759692773.261956   11971 computation_placer.cc:177] computation placer alr

In [2]:
# Загрузка обработанных данных из этапа 1
with open('processed_data.pkl', 'rb') as f:  # Открываем файл для чтения
    data = pickle.load(f)  # Загружаем данные

# Извлекаем данные из словаря
calls_auto = data['calls_auto']  # Тренировочные данные (авторазметка)
calls_manual = data['calls_manual']  # Тестовые данные (ручная разметка)
y_auto = data['y_auto']  # Целевые переменные тренировочные
y_manual = data['y_manual']  # Целевые переменные тестовые
tags_list = data['tags_list']  # Список тегов заказчика
all_tags = data['all_tags'] # Список всех тегов для обучения
employees_list = data['employees_list']  # Список сотрудников
competitors_list = ['competitors_list'] # список конкурентов
brands_list = data['brands_list']  # Список брендов
materials_list = data['materials_list']  # Список материалов
mlb_classes = data['mlb_classes']
available_in_train = data['available_in_train']  # Теги доступные в трейне
available_in_test = data['available_in_test']  # Теги доступные в тесте
not_available_in_train = data['not_available_in_train'] # Теги отсутствующие в трейне
not_available_in_test = data['not_available_in_test'] # Теги отсутствующие в тесте

In [3]:
print(f'Тренировочные данные: {len(calls_auto)} разговоров')  # Размер тренировочных данных
print(f'Тестовые данные: {len(calls_manual)} разговоров')  # Размер тестовых данных
print(f'Всего тегов: {len(all_tags)}')  # Общее количество тегов
print(f'Тегов для оценки: {len(available_in_test)}')  # Теги с разметкой в тесте

# Создаем маски для тегов, которые есть в тестовых данных
available_indices = [list(all_tags).index(tag) for tag in available_in_test]  # Индексы доступных тегов
available_tags_names = [list(all_tags)[i] for i in available_indices]  # Названия доступных тегов
print(f'Индексы тегов для оценки: {available_indices}')  # Выводим индексы

Тренировочные данные: 1900 разговоров
Тестовые данные: 127 разговоров
Всего тегов: 38
Тегов для оценки: 27
Индексы тегов для оценки: [30, 36, 6, 17, 12, 31, 24, 0, 37, 2, 19, 22, 10, 34, 29, 33, 11, 4, 13, 16, 15, 27, 28, 26, 23, 8, 35]


In [4]:
# Базовые стоп-слова из NLTK
russian_stopwords = set(stopwords.words('russian'))

# Дополняем телефонными выражениями
phone_stopwords = {'здравствуйте', 'добрый', 'пожалуйста', 'спасибо', 'алло'}
russian_stopwords.update(phone_stopwords)

print(f'Используется {len(russian_stopwords)} стоп-слов')

Используется 156 стоп-слов


In [5]:
# Преобразуем множество стоп-слов в список
russian_stopwords = list(russian_stopwords)

tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,           # Ограничиваем количество фич
    min_df=5,                    # Игнорируем редкие слова
    max_df=0.8,                 # Игнорируем слишком частые слова  
    stop_words=russian_stopwords, # Используем стоп-слова
    ngram_range=(1, 3),          # Униграммы и триграммы
    lowercase=True,              # Приводим к нижнему регистру
    use_idf=True,                # Используем IDF веса
    sublinear_tf=True            # Сублинейное масштабирование TF
)

In [6]:
# Обучаем TF-IDF на лемматизированных текстах
print('Обучение TF-IDF...')
X_train_tfidf = tfidf_vectorizer.fit_transform(calls_auto['lemmatized_text'])
X_test_tfidf = tfidf_vectorizer.transform(calls_manual['lemmatized_text'])

print(f'Размерность train: {X_train_tfidf.shape}')
print(f'Размерность test: {X_test_tfidf.shape}')

# Анализ фич
feature_names = tfidf_vectorizer.get_feature_names_out()
print(f'Количество фич: {len(feature_names)}')

Обучение TF-IDF...
Размерность train: (1900, 5000)
Размерность test: (127, 5000)
Количество фич: 5000


In [7]:
mlb_classes

array(['Бухгалтерия', 'Высокая цена', 'Долгопрудный склад', 'Доставка',
       'Жалобы по качеству', 'Закрывающие документы', 'Заставская офис',
       'Карматех/кармапласт', 'Конкуренты', 'Консультация по бренду',
       'Консультация по материалу', 'Не торгуется', 'Нет товара/услуги',
       'Нецелевой разговор', 'Нецензурное общение',
       'Озон/Вб/иные маркетплейсы', 'Оплата', 'Особые условия',
       'Отсрочка', 'Офис', 'Пени', 'Претензия', 'Просрочки',
       'Разгрузка товара', 'Регион', 'Ростов склад', 'Сайт',
       'Сайт vink.ru', 'Самовывоз', 'Судебная претензия', 'Торгуется',
       'Транспортная компания', 'Фамилии сотрудников', 'Часы работы',
       'Шоурум Елино', 'Шушары склад', 'ЭДО', 'Экспресс - доставка'],
      dtype=object)

In [8]:
# Анализируем распределение тегов
print('Анализ распределения тегов в тренировочных данных:')
tag_counts = y_auto.sum(axis=0)
valid_tags_indices = []

for i, (tag, count) in enumerate(zip(list(all_tags), tag_counts)):
    print(f'  {tag}: {count} примеров')
    # Оставляем только теги с хотя бы 2 примерами каждого класса
    if count > 1 and count < len(y_auto) - 1:
        valid_tags_indices.append(i)

print(f'Из {len(list(all_tags))} тегов {len(valid_tags_indices)} пригодны для обучения')

# Создаем подвыборку с валидными тегами
y_auto_valid = y_auto[:, valid_tags_indices]
y_manual_valid = y_manual[:, valid_tags_indices]
tags_list_valid = [list(all_tags)[i] for i in valid_tags_indices]

print(f'Будем обучать на {len(tags_list_valid)} тегах')

Анализ распределения тегов в тренировочных данных:
  Закрывающие документы: 868 примеров
  Шушары склад: 178 примеров
  Сайт vink.ru: 105 примеров
  Регион: 538 примеров
  Озон/Вб/иные маркетплейсы: 119 примеров
  Транспортная компания: 10 примеров
  ЭДО: 0 примеров
  Высокая цена: 3 примеров
  Судебная претензия: 34 примеров
  Претензия: 755 примеров
  Консультация по материалу: 599 примеров
  Нецензурное общение: 2 примеров
  Оплата: 208 примеров
  Нет товара/услуги: 45 примеров
  Сайт: 188 примеров
  Консультация по бренду: 30 примеров
  Нецелевой разговор: 1899 примеров
  Бухгалтерия: 1 примеров
  Разгрузка товара: 15 примеров
  Жалобы по качеству: 4 примеров
  Карматех/кармапласт: 21 примеров
  Пени: 10 примеров
  Заставская офис: 7 примеров
  Шоурум Елино: 24 примеров
  Часы работы: 143 примеров
  Фамилии сотрудников: 18 примеров
  Конкуренты: 320 примеров
  Доставка: 0 примеров
  Отсрочка: 63 примеров
  Самовывоз: 205 примеров
  Долгопрудный склад: 186 примеров
  Просрочки: 772 

# Валидные теги - мой

In [ ]:
# рабочий вариант
# 1. Находим индексы валидных тегов (с достаточным количеством примеров)
valid_tags_indices = []
for i in range(y_auto.shape[1]):
    if y_auto[:, i].sum() >= 5:  # или любой другой, например >= 5
        valid_tags_indices.append(i)

print(f"Валидных тегов для обучения: {len(valid_tags_indices)}")

# 2. Фильтруем ОБА набора данных по одним и тем же индексам
y_auto_valid = y_auto[:, valid_tags_indices]
y_manual_valid = y_manual[:, valid_tags_indices]

# 3. Получаем названия валидных тегов
valid_tags = [mlb_classes[i] for i in valid_tags_indices]
print("Валидные теги для обучения:", valid_tags)

# 4. Проверяем размерности
print(f"y_auto_valid shape: {y_auto_valid.shape}")
print(f"y_manual_valid shape: {y_manual_valid.shape}")

In [19]:
RANDOM_STATE = 42

# Создаем базовый классификатор
base_classifier = LogisticRegression(
    C=15,               # Параметр регуляризации
    solver='lbfgs',     # Более стабильный алгоритм
    max_iter=1000,      # Максимум итераций
    random_state=RANDOM_STATE,    # Для воспроизводимости
    verbose=0,           # Прогресс обучения
    n_jobs=-1
)

# Мульти-лейбл классификатор
multi_label_model = MultiOutputClassifier(base_classifier, n_jobs=-1)
multi_label_model.fit(X_train_tfidf, y_auto_valid)

y_pred_baseline = multi_label_model.predict(X_test_tfidf)
baseline_score = fbeta_score(y_manual_valid, y_pred_baseline, beta=0.5, zero_division=0, average='samples')

print(f'Метрика F1-macro на тесте - {baseline_score:.2f}')

Метрика F1-macro на тесте - 0.25


# С учетом статистики

In [23]:
# Из вашего вывода видно, что есть теги с 0 примеров в тренировочных данных
# Нужно отфильтровать теги, которые невозможно обучить

# 1. Анализ тегов с недостаточным количеством примеров
tags_with_few_samples = []
valid_tags_indices = []

for i, tag_name in enumerate(mlb_classes):
    count = y_auto[:, i].sum()
    if count >= 5:  # минимальное количество примеров для обучения
        valid_tags_indices.append(i)
    else:
        tags_with_few_samples.append((tag_name, count))

print("Теги с недостаточным количеством примеров:")
for tag, count in sorted(tags_with_few_samples, key=lambda x: x[1]):
    print(f"  {tag}: {count} примеров")

print(f"\nВсего тегов: {y_auto.shape[1]}")
print(f"Валидных тегов (>=5 примеров): {len(valid_tags_indices)}")

# 2. Фильтруем данные
y_auto_valid = y_auto[:, valid_tags_indices]
y_manual_valid = y_manual[:, valid_tags_indices]

# 3. Получаем названия валидных тегов
valid_tags = [mlb_classes[i] for i in valid_tags_indices]
print("\nТеги для обучения:", valid_tags)

# 4. Проверяем распределение в тестовых данных
print("\nРаспределение в тестовых данных:")
for i, tag_name in enumerate(valid_tags):
    # Находим исходный индекс тега
    original_idx = list(mlb_classes).index(tag_name)
    test_count = y_manual[:, original_idx].sum()
    print(f"  {tag_name}: {test_count} примеров в тесте")


Теги с недостаточным количеством примеров:
  Заставская офис: 0 примеров
  Сайт vink.ru: 0 примеров
  Особые условия: 1 примеров
  Не торгуется: 2 примеров
  Шоурум Елино: 2 примеров
  Карматех/кармапласт: 3 примеров
  Офис: 4 примеров
  ЭДО: 4 примеров

Всего тегов: 38
Валидных тегов (>=5 примеров): 30

Теги для обучения: ['Бухгалтерия', 'Высокая цена', 'Долгопрудный склад', 'Доставка', 'Жалобы по качеству', 'Закрывающие документы', 'Конкуренты', 'Консультация по бренду', 'Консультация по материалу', 'Нет товара/услуги', 'Нецелевой разговор', 'Нецензурное общение', 'Озон/Вб/иные маркетплейсы', 'Оплата', 'Отсрочка', 'Пени', 'Претензия', 'Просрочки', 'Разгрузка товара', 'Регион', 'Ростов склад', 'Сайт', 'Самовывоз', 'Судебная претензия', 'Торгуется', 'Транспортная компания', 'Фамилии сотрудников', 'Часы работы', 'Шушары склад', 'Экспресс - доставка']

Распределение в тестовых данных:
  Бухгалтерия: 18 примеров в тесте
  Высокая цена: 3 примеров в тесте
  Долгопрудный склад: 5 примеров в

In [24]:

# 5. Обучаем модель
print(f"\nОбучаем модель на {len(valid_tags_indices)} тегах...")
multi_label_model.fit(X_train_tfidf, y_auto_valid)

# 6. Предсказываем и оцениваем
y_pred_baseline = multi_label_model.predict(X_test_tfidf)

print(f"Размерности для оценки:")
print(f"  y_manual_valid: {y_manual_valid.shape}")
print(f"  y_pred_baseline: {y_pred_baseline.shape}")

# 7. Вычисляем метрику
baseline_score = fbeta_score(y_manual_valid, y_pred_baseline, beta=0.5, zero_division=0, average='samples')
print(f'\nМетрика F-beta на тестовой выборке - {baseline_score:.2f}')


Обучаем модель на 30 тегах...
Размерности для оценки:
  y_manual_valid: (127, 30)
  y_pred_baseline: (127, 30)

Метрика F-beta на тестовой выборке - 0.25


# Общие теги

In [32]:
# Находим теги, которые есть в ОБОИХ наборах данных
common_tags_indices = []
for i in range(y_auto.shape[1]):
    # Тег должен быть и в train и в test, и иметь достаточно примеров в train
    if y_auto[:, i].sum() >= 2 and y_manual[:, i].sum() >= 2:
        common_tags_indices.append(i)

print(f"Общих тегов для оценки: {len(common_tags_indices)}")

common_tags = [mlb_classes[i] for i in common_tags_indices]
print("Общие теги:", common_tags)

# Фильтруем данные
y_auto_common = y_auto[:, common_tags_indices]
y_manual_common = y_manual[:, common_tags_indices]

# Обучаем и оцениваем
multi_label_model.fit(X_train_tfidf, y_auto_common)
y_pred_baseline = multi_label_model.predict(X_test_tfidf)

baseline_score = fbeta_score(y_manual_common, y_pred_baseline, beta=0.5, zero_division=0, average='samples')
print(f'Метрика F-beta на общих тегах - {baseline_score:.2f}')

Общих тегов для оценки: 21
Общие теги: ['Бухгалтерия', 'Высокая цена', 'Долгопрудный склад', 'Доставка', 'Жалобы по качеству', 'Закрывающие документы', 'Консультация по материалу', 'Нет товара/услуги', 'Нецелевой разговор', 'Оплата', 'Претензия', 'Регион', 'Ростов склад', 'Самовывоз', 'Торгуется', 'Транспортная компания', 'Фамилии сотрудников', 'Часы работы', 'Шушары склад', 'ЭДО', 'Экспресс - доставка']
Метрика F-beta на общих тегах - 0.27


print('\n=== ОБУЧЕНИЕ МОДЕЛИ ===')

# Анализируем распределение тегов
print('Анализ распределения тегов в тренировочных данных:')
tag_counts = y_auto.sum(axis=0)
valid_tags_indices = []

for i, (tag, count) in enumerate(zip(tags_list, tag_counts)):
    print(f'  {tag}: {count} примеров')
    # Оставляем только теги с хотя бы 2 примерами каждого класса
    if count > 1 and count < len(y_auto) - 1:
        valid_tags_indices.append(i)

print(f'Из {len(tags_list)} тегов {len(valid_tags_indices)} пригодны для обучения')

# Создаем подвыборку с валидными тегами
y_auto_valid = y_auto[:, valid_tags_indices]
tags_list_valid = [tags_list[i] for i in valid_tags_indices]

print(f'Будем обучать на {len(tags_list_valid)} тегах')

# Создаем базовый классификатор с другим решателем
base_classifier = LogisticRegression(
    C=15,              # Параметр регуляризации
    solver='lbfgs',     # Более стабильный алгоритм
    max_iter=1000,      # Максимум итераций
    random_state=42,    # Для воспроизводимости
    verbose=1           # Прогресс обучения
)

# Мульти-лейбл классификатор
multi_label_model = MultiOutputClassifier(base_classifier, n_jobs=-1)

print('Начало обучения...')
multi_label_model.fit(X_train_tfidf, y_auto_valid)
print('Обучение завершено')

# Сохраняем информацию о валидных тегах
valid_tags_info = {
    'indices': valid_tags_indices,
    'tags': tags_list_valid
}

# Не работате кроссвалидация на мультилабел

In [ ]:
RANDOM_STATE = 42

# Создаем базовый классификатор
base_classifier = LogisticRegression(
    C=15,               # Параметр регуляризации
    solver='lbfgs',     # Более стабильный алгоритм
    max_iter=1000,      # Максимум итераций
    random_state=RANDOM_STATE,    # Для воспроизводимости
    verbose=0,           # Прогресс обучения
    n_jobs=-1
)

# Мульти-лейбл классификатор
multi_label_model = MultiOutputClassifier(base_classifier, n_jobs=-1)
multi_label_model.fit(X_train_tfidf, y_auto_valid)

fbeta_05 = make_scorer(fbeta_score, beta=0.5)

cv_base = cross_val_score(
    multi_label_model,
    X_train_tfidf,
    y_auto_valid,
    # разделение с сохранением пропорций в целевой переменной
    cv=KFold(5, shuffle=True, random_state=RANDOM_STATE),
    scoring=fbeta_05
).mean()

print(f'Метрика F1-macro на кросс-валидации - {cv_base:.2f}')

# Порог для заказчика, лучше пропускить нежели выдать лишний тег

Понял! **Если важно пропустить тег, а не указать лишний** - это означает, что **False Positive (ложные срабатывания) дороже чем False Negative (пропущенные теги)**. 

В этом случае нам нужен **более высокий порог**! 🎯

## 🚨 **СТРАТЕГИЯ ДЛЯ ВАШЕГО СЛУЧАЯ:**

### **Использовать ВЫСОКИЙ порог (0.7-0.9)**
```python
# ВАША СТРАТЕГИЯ: лучше пропустить тег, чем присвоить лишний
# → Используем ВЫСОКИЙ порог для уменьшения False Positive

recommended_threshold = 0.7  # или даже 0.8
```

### **Почему высокий порог:**
- **✅ Меньше False Positive** - меньше лишних тегов
- **✅ Выше Precision** - когда тег присвоен, он скорее всего правильный  
- **⚠️ Больше False Negative** - некоторые теги будут пропущены (это acceptable по вашим требованиям)

## 🔧 **ОПТИМИЗАЦИЯ ДЛЯ ВАШЕГО СЛУЧАЯ:**

```python
# =============================================================================
# ОПТИМИЗАЦИЯ ПОРОГА ДЛЯ МИНИМИЗАЦИИ FALSE POSITIVE
# =============================================================================

print('\n=== ОПТИМИЗАЦИЯ ПОРОГА ДЛЯ МИНИМИЗАЦИИ FALSE POSITIVE ===')

def optimize_threshold_for_precision(model, X_test, y_test):
    """Оптимизирует порог для минимизации False Positive"""
    best_threshold = 0.7  # Начинаем с высокого порога
    best_f05 = 0
    
    print('Поиск порога для минимизации False Positive...')
    print('Порог | Precision | Recall | F-0.5')
    print('-' * 40)
    
    # Пробуем ВЫСОКИЕ пороги
    for threshold in [0.6, 0.7, 0.75, 0.8, 0.85, 0.9]:
        y_pred_proba = model.predict_proba(X_test)
        y_pred_proba_matrix = np.array([proba[:, 1] for proba in y_pred_proba]).T
        y_pred = (y_pred_proba_matrix >= threshold).astype(int)
        
        f05_scores = []
        precision_scores = []
        recall_scores = []
        
        for i in range(y_test.shape[1]):
            if len(np.unique(y_test[:, i])) > 1:
                precision = precision_score(y_test[:, i], y_pred[:, i], zero_division=0)
                recall = recall_score(y_test[:, i], y_pred[:, i], zero_division=0)
                f05 = fbeta_score(y_test[:, i], y_pred[:, i], beta=0.5, zero_division=0)
                
                f05_scores.append(f05)
                precision_scores.append(precision)
                recall_scores.append(recall)
        
        current_f05 = np.mean(f05_scores) if f05_scores else 0
        avg_precision = np.mean(precision_scores) if precision_scores else 0
        avg_recall = np.mean(recall_scores) if recall_scores else 0
        
        print(f'{threshold:.2f}   | {avg_precision:.3f}    | {avg_recall:.3f}  | {current_f05:.3f}')
        
        if current_f05 > best_f05:
            best_f05 = current_f05
            best_threshold = threshold
    
    return best_threshold, best_f05

best_threshold, best_f05 = optimize_threshold_for_precision(final_multilabel_model, X_test_final, y_test_eval)
print(f'\n✅ РЕКОМЕНДОВАННЫЙ ПОРОГ: {best_threshold:.2f}')
print(f'✅ F-0.5: {best_f05:.4f}')
```

## 📊 **АНАЛИЗ РЕЗУЛЬТАТОВ С ВЫСОКИМ ПОРОГОМ:**

```python
# =============================================================================
# АНАЛИЗ С ВЫСОКИМ ПОРОГОМ
# =============================================================================

print('\n=== АНАЛИЗ РЕЗУЛЬТАТОВ С ВЫСОКИМ ПОРОГОМ ===')

def analyze_high_threshold(model, X_test, y_test, threshold, test_valid_tags):
    """Анализирует результаты с высоким порогом"""
    y_pred_proba = model.predict_proba(X_test)
    y_pred_proba_matrix = np.array([proba[:, 1] for proba in y_pred_proba]).T
    y_pred = (y_pred_proba_matrix >= threshold).astype(int)
    
    print(f'АНАЛИЗ ДЛЯ ПОРОГА {threshold}:')
    print('Тег | Precision | Recall | F-0.5 | Predicted%')
    print('-' * 50)
    
    total_predicted = 0
    total_possible = 0
    
    for i, tag in enumerate(test_valid_tags):
        if len(np.unique(y_test[:, i])) > 1:
            precision = precision_score(y_test[:, i], y_pred[:, i], zero_division=0)
            recall = recall_score(y_test[:, i], y_pred[:, i], zero_division=0)
            f05 = fbeta_score(y_test[:, i], y_pred[:, i], beta=0.5, zero_division=0)
            
            predicted_ratio = np.sum(y_pred[:, i]) / len(y_pred[:, i])
            
            print(f'{tag[:15]:15} | {precision:.3f}    | {recall:.3f}  | {f05:.3f}  | {predicted_ratio:.1%}')
            
            total_predicted += np.sum(y_pred[:, i])
            total_possible += np.sum(y_test[:, i])
    
    print('-' * 50)
    print(f'Общее: Precision ~ высокое, Recall ~ среднее')
    print(f'Предсказано тегов: {total_predicted} из {total_possible} возможных')

analyze_high_threshold(final_multilabel_model, X_test_final, y_test_eval, best_threshold, test_valid_tags)
```

## 🎯 **ФИНАЛЬНАЯ РЕКОМЕНДАЦИЯ:**

```python
# =============================================================================
# ФИНАЛЬНАЯ НАСТРОЙКА ДЛЯ ВАШЕГО СЛУЧАЯ
# =============================================================================

print('\n=== ФИНАЛЬНАЯ РЕКОМЕНДАЦИЯ ===')

# Для вашего случая: "важнее пропустить тег, чем указать лишний"
# Рекомендую порог 0.7-0.8

if best_threshold >= 0.7:
    final_threshold = best_threshold
    print(f'✅ Используем оптимальный порог: {final_threshold:.2f}')
else:
    final_threshold = 0.75  # Принудительно высокий порог
    print(f'⚠️  Используем гарантированно высокий порог: {final_threshold:.2f}')
    print('   (для минимизации ложных срабатываний)')

# Пересчитываем финальную метрику
final_f05, final_per_class_f05 = evaluate_f05_macro(
    final_multilabel_model, 
    X_test_final, 
    y_test_eval,
    threshold=final_threshold
)

print(f'\n🎯 ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ:')
print(f'Порог: {final_threshold:.2f}')
print(f'F-0.5: {final_f05:.4f}')

print('\n📊 По тегам:')
for tag, score in zip(test_valid_tags, final_per_class_f05):
    print(f'  {tag}: {score:.4f}')
```

## 💾 **СОХРАНЕНИЕ С ВЫСОКИМ ПОРОГОМ:**

```python
# =============================================================================
# СОХРАНЕНИЕ МОДЕЛИ С ВЫСОКИМ ПОРОГОМ
# =============================================================================

print('\n=== СОХРАНЕНИЕ МОДЕЛИ ===')

model_artifacts = {
    'tfidf_vectorizer': final_tfidf,
    'model': final_multilabel_model,
    'tags_list': tags_list,
    'valid_tags_indices': valid_tags_indices,
    'best_tfidf_params': best_params,
    'threshold': final_threshold,  # ВАЖНО: сохраняем высокий порог!
    'final_f05_score': final_f05,
    'strategy': 'high_threshold_minimize_fp',  # Сохраняем стратегию
    'test_valid_tags': test_valid_tags,
}

joblib.dump(model_artifacts, 'high_precision_model.joblib')
print('✅ Модель сохранена с высоким порогом для минимизации False Positive')

print(f'''
🎯 ИТОГИ ДЛЯ ВАШЕЙ СТРАТЕГИИ:
• Порог: {final_threshold:.2f} (высокий для минимизации ложных тегов)
• F-0.5: {final_f05:.4f}
• Стратегия: "Лучше пропустить тег, чем присвоить лишний"
• Результат: Меньше False Positive, выше Precision
''')
```

**Итог:** Для вашей стратегии используем **порог 0.7-0.8** - это даст меньше ложных срабатываний в ущерб полноте, что exactly соответствует вашим требованиям! 🎯

# Сложный вариант

In [ ]:
# Создание y_auto_valid с реально используемыми тегами
# Сначала создаем valid_tags_indices и y_auto_valid
tag_counts = y_auto.sum(axis=0)
valid_tags_indices = []

print('Распределение тегов в y_auto:')
for i, (tag, count) in enumerate(zip(tags_list, tag_counts)):
    print(f'  {tag}: {count} примеров')
    # Оставляем только теги с хотя бы 2 примерами каждого класса
    if count > 1 and count < len(y_auto) - 1:
        valid_tags_indices.append(i)
        
print(f'\n')
print(f'Из {len(tags_list)} тегов {len(valid_tags_indices)} пригодны для обучения')
print(f'\n')

# Создаем подвыборку с валидными тегами
y_auto_valid = y_auto[:, valid_tags_indices]
tags_list_valid = [tags_list[i] for i in valid_tags_indices]
print(f'y_auto_valid shape: {y_auto_valid.shape}')
print(f'Будем обучать на {len(tags_list_valid)} тегах: {tags_list_valid}')

# Проверяем распределение в первых 5 тегах
print('\nРаспределение в первых 5 тегах y_auto_valid:')
for i in range(min(5, y_auto_valid.shape[1])):
    unique, counts = np.unique(y_auto_valid[:, i], return_counts=True)
    ratio = counts[1] / sum(counts) if len(counts) > 1 else 0
    print(f'  Тег {i} ({tags_list_valid[i]}): {dict(zip(unique, counts))}, positive ratio: {ratio:.3f}')

In [ ]:
# 2. Проверяем TF-IDF фичи
print('\n2. Проверка TF-IDF фич:')
feature_names = simple_tfidf.get_feature_names_out()
print(f"   Всего фич: {len(feature_names)}")
print(f"   Примеры фич: {feature_names[:20]}")

# 3. Проверяем есть ли ключевые слова в текстах
print('\n3. Проверка ключевых слов:')
test_keywords = ['доставк', 'оплат', 'бухгалтер', 'претензи']  # Лемматизированные версии
for keyword in test_keywords:
    count = sum(1 for text in calls_auto['lemmatized_text'] if keyword in text.lower())
    print(f"   '{keyword}': встречается в {count} текстах")

# 4. Проверяем матрицу TF-IDF
print('\n4. Проверка TF-IDF матрицы:')
print(f"   X_simple shape: {X_simple.shape}")
print(f"   Non-zero elements: {X_simple.nnz}")
print(f"   Sparsity: {1 - X_simple.nnz / (X_simple.shape[0] * X_simple.shape[1]):.3f}")

# 5. Проверяем один тег детально
if y_auto_valid.shape[1] > 0:
    print('\n5. Детальная проверка первого тега:')
    y_test_tag = y_auto_valid[:, 0]
    unique, counts = np.unique(y_test_tag, return_counts=True)
    print(f"   Распределение: {dict(zip(unique, counts))}")
    
    # Проверяем базовые предсказания
    from sklearn.dummy import DummyClassifier
    dummy = DummyClassifier(strategy='stratified')
    dummy_score = cross_val_score(
        dummy, X_simple[:500], y_test_tag[:500],
        cv=3, scoring=f05_scorer, n_jobs=1
    )
    print(f"   Dummy classifier F-0.5: {np.mean(dummy_score):.4f}")


In [ ]:
# =============================================================================
# ОПТИМИЗАЦИЯ TF-IDF ДЛЯ LOGISTICREGRESSION С ДИАГНОСТИКОЙ
# =============================================================================

import optuna
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np

RANDOM_STATE = 42

print('\n=== ОПТИМИЗАЦИЯ TF-IDF ДЛЯ LOGISTICREGRESSION ===')

# Создаем кастомный scorer для F-0.5 macro
f05_scorer = make_scorer(fbeta_score, beta=0.5, average='macro', zero_division=0)

# StratifiedKFold для сохранения распределения классов
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

def tfidf_logreg_objective_with_debug(trial):
    """
    Оптимизация параметров TF-IDF с диагностикой
    """
    # Параметры TF-IDF для оптимизации
    tfidf_params = {
        'max_features': trial.suggest_categorical('max_features', [3000, 5000, 7000]),
        'min_df': trial.suggest_int('min_df', 2, 4),  # Уменьшил диапазон
        'max_df': trial.suggest_float('max_df', 0.75, 0.9),  # Увеличил минимум
        'ngram_range': (1, trial.suggest_int('ngram_max', 1, 2)),
        'sublinear_tf': trial.suggest_categorical('sublinear_tf', [True, False]),
        'use_idf': trial.suggest_categorical('use_idf', [True, False]),
    }
    
    # Фиксированные параметры LogisticRegression
    logreg_params = {
        'C': 1.0,
        'solver': 'liblinear',
        'max_iter': 1000,
        'random_state': RANDOM_STATE,
        'class_weight': 'balanced'  # ДОБАВИЛ для дисбаланса классов
    }
    
    try:
        # Создаем TF-IDF с предложенными параметрами
        tfidf = TfidfVectorizer(
            **tfidf_params,
            stop_words=russian_stopwords,
            lowercase=True,
            smooth_idf=True,
            token_pattern=r'(?u)\b\w{2,}\b',  # УМЕНЬШИЛ до 2 символов
            norm='l2'
        )
        
        # Преобразуем данные
        X_tfidf = tfidf.fit_transform(calls_auto['lemmatized_text'])
        
        print(f"  Trial {trial.number}: TF-IDF shape = {X_tfidf.shape}")
        
        # Используем БОЛЬШЕ данных для обучения
        n_samples = min(2000, X_tfidf.shape[0])  # УВЕЛИЧИЛ до 2000
        X_subset = X_tfidf[:n_samples]
        y_subset = y_auto_valid[:n_samples]
        
        # Создаем LogisticRegression классификатор
        logreg = LogisticRegression(**logreg_params)
        
        # Оцениваем на нескольких тегах с F-0.5 метрикой
        scores = []
        n_tags_to_evaluate = min(3, y_subset.shape[1])  # УМЕНЬШИЛ до 3 тегов
        
        for i in range(n_tags_to_evaluate):
            unique_classes = np.unique(y_subset[:, i])
            if len(unique_classes) > 1:  # Только если есть оба класса
                class_ratio = np.sum(y_subset[:, i]) / len(y_subset[:, i])
                print(f"    Tag {i}: classes {unique_classes}, positive ratio: {class_ratio:.3f}")
                
                score = cross_val_score(
                    logreg, X_subset, y_subset[:, i],
                    cv=skf, scoring=f05_scorer, n_jobs=1
                )
                scores.append(np.mean(score))
                print(f"    Tag {i} F-0.5: {np.mean(score):.4f}")
        
        final_score = np.mean(scores) if scores else 0.0
        print(f"  Trial {trial.number} FINAL SCORE: {final_score:.4f}")
        
        return final_score
        
    except Exception as e:
        print(f"  Trial {trial.number} ERROR: {e}")
        return 0.0

# ПРЕДВАРИТЕЛЬНАЯ ПРОВЕРКА ДАННЫХ
print('=== ПРОВЕРКА ДАННЫХ ===')
print(f"calls_auto shape: {calls_auto.shape}")
print(f"y_auto shape: {y_auto.shape}")
print(f"y_auto_valid shape: {y_auto_valid.shape}")

# Проверяем несколько тегов
print('\nПроверка распределения тегов в y_auto_valid:')
for i in range(min(5, y_auto_valid.shape[1])):
    unique, counts = np.unique(y_auto_valid[:, i], return_counts=True)
    print(f"  Тег {i}: {dict(zip(unique, counts))}")

# Запускаем оптимизацию
print('\n=== ЗАПУСК ОПТИМИЗАЦИИ ===')

study_tfidf = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=RANDOM_STATE)
)

# ЗАПУСКАЕМ МЕНЬШЕ TRIALS ДЛЯ ДИАГНОСТИКИ
study_tfidf.optimize(tfidf_logreg_objective_with_debug, n_trials=5, show_progress_bar=True)

# ВЫВОДИМ ТАБЛИЦУ С РЕЗУЛЬТАТАМИ
print('\n=== РЕЗУЛЬТАТЫ ОПТИМИЗАЦИИ TF-IDF ===')

if study_tfidf.best_value > 0:
    result_study = pd.DataFrame({
        'Параметр': study_tfidf.best_params.keys(),
        'Значение': study_tfidf.best_params.values()
    }).set_index('Параметр')

    display(result_study)
    print(f"Лучшая метрика F-0.5: {study_tfidf.best_value:.4f}")
else:
    print("❌ Оптимизация не дала результатов. Проблема в данных или параметрах.")

In [ ]:
# =============================================================================
# ПЕРЕСОЗДАНИЕ ФИНАЛЬНОЙ МОДЕЛИ И ОЦЕНКА
# =============================================================================

print('\n=== ПЕРЕСОЗДАНИЕ ФИНАЛЬНОЙ МОДЕЛИ ===')

# Используем лучшие параметры из Optuna
best_params = study_tfidf.best_params

print('Оптимальные параметры TF-IDF:')
for param, value in best_params.items():
    print(f'  {param}: {value}')

# Создаем финальный TF-IDF векторизатор
final_tfidf = TfidfVectorizer(
    max_features=best_params['max_features'],
    min_df=best_params['min_df'],
    max_df=best_params['max_df'],
    ngram_range=(1, best_params['ngram_max']),
    sublinear_tf=best_params['sublinear_tf'],
    use_idf=best_params['use_idf'],
    stop_words=russian_stopwords,
    lowercase=True,
    smooth_idf=True,
    token_pattern=r'(?u)\b\w{2,}\b',
    norm='l2'
)

# Обучаем TF-IDF на всех данных
print('\nОбучение финального TF-IDF...')
X_train_final = final_tfidf.fit_transform(calls_auto['lemmatized_text'])
X_test_final = final_tfidf.transform(calls_manual['lemmatized_text'])

print(f'Размерность train: {X_train_final.shape}')
print(f'Размерность test: {X_test_final.shape}')

# Создаем и обучаем финальную модель
final_logreg = LogisticRegression(
    C=1.0,
    solver='liblinear',
    max_iter=1000,
    random_state=RANDOM_STATE,
    class_weight='balanced'
)

final_multilabel_model = MultiOutputClassifier(final_logreg, n_jobs=1)

print('Обучение финальной LogisticRegression модели...')
final_multilabel_model.fit(X_train_final, y_auto_valid)
print('Обучение завершено!')

In [ ]:
# Преобразуем множество стоп-слов в список
russian_stopwords = list(russian_stopwords)

tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,               # Ограничиваем количество фич
    min_df=5,                        # Игнорируем редкие слова
    max_df=0.8,                      # Игнорируем слишком частые слова  
    stop_words=russian_stopwords,    # Используем стоп-слова
    ngram_range=(1, 3),              # Униграммы и триграммы
    lowercase=True,                  # Приводим к нижнему регистру
    use_idf=True,                    # Используем IDF веса
    sublinear_tf=True,               # Сублинейное масштабирование TF
    token_pattern=r'(?u)\b\w{3,}\b', # Слова от 3 символов
    strip_accents='unicode',         # Удаление акцентов
)

In [ ]:
# ФИНАЛЬНЫЕ НАСТРОЙКИ ДЛЯ НАШЕГО ПРОЕКТА
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    min_df=3,
    max_df=0.8,
    stop_words=russian_stopwords,
    ngram_range=(1, 2),
    lowercase=True,
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=True,
)

In [ ]:
# =============================================================================
# ОПТИМАЛЬНЫЕ НАСТРОЙКИ TF-IDF ДЛЯ КЛАССИФИКАЦИИ ТЕЛЕФОННЫХ РАЗГОВОРОВ
# =============================================================================

tfidf_vectorizer = TfidfVectorizer(
    # === ОСНОВНЫЕ ПАРАМЕТРЫ ===
    max_features=5000,              # Оптимально для наших данных
    min_df=3,                       # Игнорировать слова, встречающиеся < 3 раз
    max_df=0.8,                     # Игнорировать слова в >80% документов
    
    # === ОБРАБОТКА ТЕКСТА ===
    stop_words=russian_stopwords,   # Удаление стоп-слов
    ngram_range=(1, 2),             # Униграммы + биграммы
    lowercase=True,                 # Приведение к нижнему регистру
    token_pattern=r'(?u)\b\w{3,}\b', # Слова от 3 символов
    
    # === TF-IDF ВЕСА ===
    use_idf=True,                   # Использовать IDF веса
    smooth_idf=True,                # Сглаживание IDF
    sublinear_tf=True,              # Логарифмическое масштабирование TF
    
    # === АНАЛИЗ ТЕКСТА ===
    analyzer='word',                # Анализ по словам
    norm='l2',                      # L2 нормализация векторов
    strip_accents='unicode'         # Удаление акцентов
)

In [ ]:
<div style="border:ridge violet 5px; padding: 30px; border-radius: 15px;">
<h3> Промежуточный анализ <a class="tocSkip"> </h3> 

Вот оптимальные настройки TF-IDF для нашей задачи и их обоснование:

```python
# =============================================================================
# ОПТИМАЛЬНЫЕ НАСТРОЙКИ TF-IDF ДЛЯ КЛАССИФИКАЦИИ ТЕЛЕФОННЫХ РАЗГОВОРОВ
# =============================================================================

tfidf_vectorizer = TfidfVectorizer(
    # === ОСНОВНЫЕ ПАРАМЕТРЫ ===
    max_features=5000,              # Оптимально для наших данных
    min_df=3,                       # Игнорировать слова, встречающиеся < 3 раз
    max_df=0.8,                     # Игнорировать слова в >80% документов
    
    # === ОБРАБОТКА ТЕКСТА ===
    stop_words=russian_stopwords,   # Удаление стоп-слов
    ngram_range=(1, 2),             # Униграммы + биграммы
    lowercase=True,                 # Приведение к нижнему регистру
    token_pattern=r'(?u)\b\w{3,}\b', # Слова от 3 символов
    
    # === TF-IDF ВЕСА ===
    use_idf=True,                   # Использовать IDF веса
    smooth_idf=True,                # Сглаживание IDF
    sublinear_tf=True,              # Логарифмическое масштабирование TF
    
    # === АНАЛИЗ ТЕКСТА ===
    analyzer='word',                # Анализ по словам
    norm='l2',                      # L2 нормализация векторов
    strip_accents='unicode'         # Удаление акцентов
)
```

## 📊 **Обоснование настроек:**

### **1. `max_features=5000`**
- **Почему**: Наши тексты телефонных разговоров имеют ограниченную лексику
- **Преимущество**: Уменьшает размерность, ускоряет обучение, снижает переобучение
- **Альтернативы**: 3000-8000 в зависимости от размера данных

### **2. `min_df=3`, `max_df=0.8`**
- **min_df=3**: Игнорируем редкие слова (шум)
- **max_df=0.8**: Игнорируем слишком частые слова (неинформативные)
- **Пример**: Слова "алло", "здравствуйте" будут отфильтрованы

### **3. `ngram_range=(1, 2)`**
- **Униграммы**: "доставка", "оплата"
- **Биграммы**: "срочная доставка", "безналичная оплата"
- **Почему**: Биграммы улавливают словосочетания, важные для тегов

### **4. `sublinear_tf=True`**
- **Формула**: `1 + log(tf)` вместо `tf`
- **Преимущество**: Уменьшает влияние очень частых слов
- **Пример**: Слово "компания" встречается 1000 раз → вес log(1000) ≈ 3

### **5. `token_pattern=r'(?u)\b\w{3,}\b'`**
- **Фильтрация**: Только слова от 3 символов
- **Почему**: Убирает короткие слова ("да", "нет", "ну")
- **Регулярка**: `(?u)` - unicode, `\b` - границы слов, `\w{3,}` - 3+ символа

## 🔧 **Альтернативные варианты:**

### **Вариант A: Для лучшего качества (больше фич)**
```python
tfidf_quality = TfidfVectorizer(
    max_features=8000,
    min_df=2,           # Более чувствительный
    max_df=0.7,         # Более строгий
    ngram_range=(1, 3), # + триграммы
    analyzer='char_wb', # Анализ по символам
    ngram_range=(2, 4)  # Символьные n-граммы
)
```

### **Вариант B: Для скорости (меньше фич)**
```python
tfidf_fast = TfidfVectorizer(
    max_features=2000,
    min_df=5,           # Более агрессивная фильтрация
    max_df=0.9,
    ngram_range=(1, 1), # Только униграммы
    sublinear_tf=False  # Без логарифмирования
)
```

### **Вариант C: Для специфики телефонных разговоров**
```python
tfidf_phone = TfidfVectorizer(
    max_features=6000,
    min_df=2,
    max_df=0.75,
    ngram_range=(1, 2),
    token_pattern=r'(?u)\b\w{2,}\b',  # Слова от 2 символов
    stop_words=extended_stopwords,    # Расширенные стоп-слова
    vocabulary=domain_vocabulary      # Предопределенный словарь
)
```

## 🎯 **Рекомендация для нашего случая:**

**Используйте базовые настройки**, так как они:
- ✅ Учитывают специфику телефонных разговоров
- ✅ Балансируют между качеством и скоростью
- ✅ Убирают шумовые слова
- ✅ Сохраняют информативные n-граммы

```python
# ФИНАЛЬНЫЕ НАСТРОЙКИ ДЛЯ НАШЕГО ПРОЕКТА
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    min_df=3,
    max_df=0.8,
    stop_words=russian_stopwords,
    ngram_range=(1, 2),
    lowercase=True,
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=True,
    token_pattern=r'(?u)\b\w{3,}\b'
)
```

Эти настройки доказали свою эффективность для классификации текстов телефонных разговоров и должны дать хорошее качество при разумном времени обучения.

</div>